<a href="https://colab.research.google.com/github/ravin235/groupc-quiz/blob/main/3_Linear_Classification_and_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Linear Classification and Logistic Regression

In [1]:
#importing library for data preprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae') 

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Armenia,1992,1,AreaPerCap,1.402924e-01,1.995463e-01,0.097188051,3.688847e-02,2.931995e-02,0.000000e+00,5.032351e-01,3A
1,Armenia,1992,1,AreaTotHA,4.830000e+05,6.870000e+05,334600,1.270000e+05,1.009430e+05,0.000000e+00,1.732543e+06,3A
2,Armenia,1992,1,BiocapPerCap,1.598044e-01,1.352610e-01,0.084003213,1.374213e-02,3.339780e-02,0.000000e+00,4.262086e-01,3A
3,Armenia,1992,1,BiocapTotGHA,5.501762e+05,4.656780e+05,289207.1078,4.731155e+04,1.149823e+05,0.000000e+00,1.467355e+06,3A
4,Armenia,1992,1,EFConsPerCap,3.875102e-01,1.894622e-01,1.26E-06,4.164833e-03,3.339780e-02,1.114093e+00,1.728629e+00,3A
...,...,...,...,...,...,...,...,...,...,...,...,...
72181,World,2016,5001,BiocapTotGHA,3.984702e+09,1.504757e+09,5111762779.0,1.095445e+09,4.726163e+08,0.000000e+00,1.216928e+10,3A
72182,World,2016,5001,EFConsPerCap,5.336445e-01,1.402092e-01,0.273495,8.974253e-02,6.329435e-02,1.646235e+00,2.746619e+00,3A
72183,World,2016,5001,EFConsTotGHA,3.984702e+09,1.046937e+09,2042179333.0,6.701039e+08,4.726163e+08,1.229237e+10,2.050891e+10,3A
72184,World,2016,5001,EFProdPerCap,5.336445e-01,1.402092e-01,0.273495,8.974253e-02,6.329435e-02,1.646235e+00,2.746619e+00,3A


In [4]:
df['QScore']

0        3A
1        3A
2        3A
3        3A
4        3A
         ..
72181    3A
72182    3A
72183    3A
72184    3A
72185    3A
Name: QScore, Length: 72186, dtype: object

In [5]:
#check distribution of target variable
df['QScore'].value_counts()

3A    51481
2A    10576
2B    10096
1A       16
1B       16
Name: QScore, dtype: int64

In [6]:
df.isna().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

In [7]:
#for simplicity, we will drop the rows with missing values.
df = df.dropna()

In [8]:
df.isna().sum()

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

In [9]:
df['QScore']

0        3A
1        3A
2        3A
3        3A
4        3A
         ..
72181    3A
72182    3A
72183    3A
72184    3A
72185    3A
Name: QScore, Length: 51713, dtype: object

In [10]:
df['QScore'].value_counts()

3A    51473
2A      224
1A       16
Name: QScore, dtype: int64

In [11]:
#An obvious change in our target variable after removing the missing values is that there are only three classes left 
#and from the distribution of the 3 classes, we can see that
#there is an obvious imbalance between the classes. #There are methods that can be applied to
#handle this imbalance such as oversampling and undersampling.
#Oversampling involves increasing the number of instances in the class with fewer instances while undersampling 
#involves reducing the data points in the class with more instances.
#For now, we will convert this to a binary classification problem by combining class '2A'and '1A'.
df['QScore'] = df['QScore'].replace(['1A'], '2A')
df['QScore'].value_counts()

<ipython-input-11-98a49813def8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['QScore'] = df['QScore'].replace(['1A'], '2A')


3A    51473
2A      240
Name: QScore, dtype: int64

In [12]:
df_2A = df[df.QScore=='2A']
df_3A = df[df.QScore=='3A'].sample(350)
data_df = df_2A.append(df_3A)
#data_df

In [13]:
import sklearn.utils
data_df = sklearn.utils.shuffle(data_df)
data_df = data_df.reset_index(drop=True)
data_df

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Georgia,2016,73,EFConsTotGHA,1.485371e+06,9.075014e+05,560941.9239,1.205618e+05,1.847296e+05,5.070554e+06,8.329659e+06,2A
1,Georgia,2016,73,EFProdPerCap,1.506440e-01,2.321877e-01,0.068249791,2.650070e-02,4.705996e-02,8.954236e-01,1.420066e+00,2A
2,Zambia,1992,251,AreaTotHA,2.800000e+06,1.826500e+07,52466800.0,9.220000e+05,3.643770e+05,0.000000e+00,7.481818e+07,3A
3,Nepal,1980,149,BiocapPerCap,2.729088e-01,9.164762e-02,0.142514891,1.065417e-02,4.153062e-02,0.000000e+00,5.592561e-01,3A
4,Georgia,2016,73,AreaPerCap,1.138880e-01,4.959846e-01,0.719007712,6.819670e-02,3.557768e-02,0.000000e+00,1.432655e+00,2A
...,...,...,...,...,...,...,...,...,...,...,...,...
585,French Guiana,2016,69,EFProdPerCap,9.140566e-02,6.791892e-02,0.429341295,1.054636e-01,5.091948e-03,8.918139e-01,1.591035e+00,2A
586,Guadeloupe,1970,87,EFProdTotGHA,1.041064e+05,7.986658e+03,9479.108474,4.868866e+04,1.021148e+03,1.090807e+05,2.803627e+05,3A
587,Cuba,1989,49,EFConsTotGHA,6.372923e+06,1.439840e+06,2267907.82,1.455596e+06,3.712708e+05,1.748190e+07,2.938943e+07,3A
588,Hungary,2016,97,BiocapTotGHA,1.497653e+07,6.165292e+05,6860071.071,4.741962e+04,1.559617e+06,0.000000e+00,2.406017e+07,2A


In [14]:
data_df["QScore"].value_counts()

3A    350
2A    240
Name: QScore, dtype: int64

In [15]:
#more preprocessing
data_df = data_df.drop(columns=['country_code', 'country', 'year'])
X = data_df.drop(columns='QScore')
y = data_df['QScore']

In [16]:
#split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
y_train.value_counts()

3A    244
2A    169
Name: QScore, dtype: int64

In [17]:
data_df

,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,EFConsTotGHA,1.485371e+06,9.075014e+05,560941.9239,1.205618e+05,1.847296e+05,5.070554e+06,8.329659e+06,2A
1,EFProdPerCap,1.506440e-01,2.321877e-01,0.068249791,2.650070e-02,4.705996e-02,8.954236e-01,1.420066e+00,2A
2,AreaTotHA,2.800000e+06,1.826500e+07,52466800.0,9.220000e+05,3.643770e+05,0.000000e+00,7.481818e+07,3A
3,BiocapPerCap,2.729088e-01,9.164762e-02,0.142514891,1.065417e-02,4.153062e-02,0.000000e+00,5.592561e-01,3A
4,AreaPerCap,1.138880e-01,4.959846e-01,0.719007712,6.819670e-02,3.557768e-02,0.000000e+00,1.432655e+00,2A
...,...,...,...,...,...,...,...,...,...
585,EFProdPerCap,9.140566e-02,6.791892e-02,0.429341295,1.054636e-01,5.091948e-03,8.918139e-01,1.591035e+00,2A
586,EFProdTotGHA,1.041064e+05,7.986658e+03,9479.108474,4.868866e+04,1.021148e+03,1.090807e+05,2.803627e+05,3A
587,EFConsTotGHA,6.372923e+06,1.439840e+06,2267907.82,1.455596e+06,3.712708e+05,1.748190e+07,2.938943e+07,3A
588,BiocapTotGHA,1.497653e+07,6.165292e+05,6860071.071,4.741962e+04,1.559617e+06,0.000000e+00,2.406017e+07,2A


In [18]:
x_train.record

285    EFConsPerCap
113    EFProdPerCap
18     BiocapTotGHA
76     EFProdTotGHA
206    BiocapPerCap
           ...     
277    EFConsTotGHA
9      EFProdTotGHA
359    EFConsTotGHA
192    EFProdPerCap
559       AreaTotHA
Name: record, Length: 413, dtype: object

In [19]:
x_test.record

225    EFProdTotGHA
14       AreaPerCap
85     EFProdTotGHA
418       AreaTotHA
132    EFConsTotGHA
           ...     
346    EFConsPerCap
369    EFProdTotGHA
140    BiocapPerCap
533    EFConsTotGHA
171    BiocapPerCap
Name: record, Length: 177, dtype: object

In [20]:
#There is still an imbalance in the class distribution. For this, we use SMOTE only on the training data to handle this.
#encode categorical variable
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train.record = encoder.fit_transform(x_train.record)
x_test.record = encoder.transform(x_test.record)

!pip install imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
x_train_balanced, y_balanced = smote.fit_resample(x_train, y_train)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=['record']))
normalised_train_df = pd.DataFrame(normalised_train_df, columns=x_train_balanced.drop(columns=['record']).columns)
normalised_train_df['record'] = x_train_balanced['record']

x_test = x_test.reset_index(drop=True)
normalised_test_df = scaler.transform(x_test.drop(columns=['record']))
normalised_test_df = pd.DataFrame(normalised_test_df, columns=x_test.drop(columns=['record']).columns)
normalised_test_df['record'] = x_test['record']

In [22]:
normalised_train_df, normalised_test_df['record']

(        crop_land  grazing_land   forest_land  fishing_ground  built_up_land  \
 0    6.811963e-10  1.202581e-10  3.816384e-10    5.301379e-11   8.602405e-10   
 1    1.396360e-09  1.816305e-12  5.198384e-11    1.103842e-12   5.540058e-10   
 2    2.241541e-03  8.548305e-05  4.509771e-03    1.805230e-03   8.118857e-03   
 3    1.328307e-04  1.292951e-05  4.426524e-05    0.000000e+00   3.110102e-04   
 4    2.745920e-11  1.911970e-12  3.496315e-11    4.645616e-10   7.615204e-12   
 ..            ...           ...           ...             ...            ...   
 483  8.119186e-04  3.023243e-03  1.276780e-04    5.991516e-04   1.506540e-03   
 484  1.909617e-03  1.084288e-03  5.656260e-04    2.302670e-03   2.893094e-03   
 485  2.124246e-10  2.713226e-11  1.500257e-10    7.970661e-12   4.029248e-10   
 486  2.054939e-04  1.469402e-05  1.615211e-05    2.217735e-04   9.658866e-04   
 487  2.180417e-04  9.645616e-06  3.657373e-04    3.791293e-05   3.332248e-04   
 
            carbon        

In [23]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)
#returns
LogisticRegression(C = 1.0, class_weight = None, dual = False, fit_intercept = True, intercept_scaling = 1, l1_ratio = None,
                   max_iter=100,multi_class='auto', n_jobs=None, penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start = False)

LogisticRegression()

### Measuring Classification Performance
##### - Cross-validation and accuracy

In [24]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg, normalised_train_df, y_balanced, cv=5, scoring='f1_macro')
scores
#prints: array([0.55594592, 0.4733312 , 0.55651249, 0.5245098 , 0.58315241])


array([0.58909853, 0.47516539, 0.47865891, 0.54975659, 0.49138577])

##### - K-Fold Cross Validation

In [25]:
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.split(normalised_train_df)
f1_scores = []
#run for every split
for train_index, test_index in kf.split(normalised_train_df):
    x_train, x_test = normalised_train_df.iloc[train_index], normalised_train_df.iloc[test_index]
    y_train, y_test = y_balanced[train_index],y_balanced[test_index]
model = LogisticRegression().fit(x_train, y_train)
#save result to list
f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), pos_label='2A')*100)

In [26]:
f1_scores 

[0.0]

##### - Stratified K-Fold Cross Validation

In [40]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
f1_scores = []
#run for every split
for train_index, test_index in skf.split(normalised_train_df, y_balanced):
    x_train, x_test = np.array(normalised_train_df)[train_index],np.array(normalised_train_df)[test_index]
    y_train, y_test = y_balanced[train_index], y_balanced[test_index]
    model = LogisticRegression().fit(x_train, y_train)
#save result to list
    f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), pos_label='2A'))
f1_scores

[0.6299212598425196,
 0.5660377358490565,
 0.6178861788617885,
 0.5825242718446602,
 0.5510204081632653]

In [28]:
# x_train, x_test, y_train, y_test, f1_scores 

##### - Leave One Out Cross Validation (LOOCV)

In [29]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(LogisticRegression(), normalised_train_df, y_balanced, cv=loo,scoring='f1_macro')
average_score = scores.mean() * 100
average_score

53.68852459016394

##### - Confusion Matrix

In [61]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
new_predictions = log_reg.predict(x_test)
cnf_mat = confusion_matrix(y_true=y_test, y_pred=new_predictions, labels=['2A','3A'])
cnf_mat #prints array([[ 35, 34],[ 50, 58]])

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([[27, 22],
       [22, 26]])

##### - Accuracy

In [62]:
accuracy = accuracy_score(y_true=y_test, y_pred=new_predictions)
print('Accuracy: {}'.format(round(accuracy*100), 2)) #prints 53.0

Accuracy: 55


##### - Precision

In [63]:
precision = precision_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('Precision: {}'.format(round(precision*100), 2)) #prints 41.0


Precision: 55


##### - Recall

In [64]:
recall = recall_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('Recall: {}'.format(round(recall*100), 2)) #prints 51.0


Recall: 55


##### - F1-Score

In [65]:
f1 = f1_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('F1: {}'.format(round(f1*100), 2)) #prints 45.0

F1: 55


### Tree-Based Methods and The Support Vector Machine

In [66]:
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(normalised_train_df, y_balanced)

DecisionTreeClassifier()